In [1]:
import tensorflow as tf
import numpy as np
import random

def set_seed(seed_value=42):
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)

set_seed(42)  # Set seed for reproducibility

2024-09-07 05:02:39.217411: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-07 05:02:39.217456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-07 05:02:39.219023: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-07 05:02:39.228449: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

def attention_block(x, g, inter_channel):
    theta_x = layers.Conv2D(inter_channel, (1, 1), strides=(1, 1), padding='same')(x)
    phi_g = layers.Conv2D(inter_channel, (1, 1), strides=(1, 1), padding='same')(g)
    add_xg = layers.add([theta_x, phi_g])
    act_xg = layers.Activation('relu')(add_xg)
    psi = layers.Conv2D(1, (1, 1), strides=(1, 1), padding='same')(act_xg)
    psi = layers.Activation('sigmoid')(psi)
    upsample_psi = layers.UpSampling2D(size=(x.shape[1] // psi.shape[1], x.shape[2] // psi.shape[2]))(psi)
    attn = layers.multiply([upsample_psi, x])
    return attn

def attention_unet(input_shape=(224, 224, 3)):
    inputs = Input(input_shape)

    # Contracting Path
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Expansive Path with Attention
    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    att6 = attention_block(c4, u6, 512)
    u6 = layers.concatenate([u6, att6])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    att7 = attention_block(c3, u7, 256)
    u7 = layers.concatenate([u7, att7])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    att8 = attention_block(c2, u8, 128)
    u8 = layers.concatenate([u8, att8])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    att9 = attention_block(c1, u9, 64)
    u9 = layers.concatenate([u9, att9])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [5]:
model = attention_unet(input_shape=(224, 224, 3))

# Compile the model with Adam optimizer (with momentum via beta_1)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_31 (Conv2D)          (None, 224, 224, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 conv2d_32 (Conv2D)          (None, 224, 224, 64)         36928     ['conv2d_31[0][0]']           
                                                                                                  
 max_pooling2d_4 (MaxPoolin  (None, 112, 112, 64)         0         ['conv2d_32[0][0]']           
 g2D)                                                                                       

In [6]:
import numpy as np

train_images = np.load("../../numpy_arr_data/train_img_224.npy")
train_masks = np.load("../../numpy_arr_data/train_mask_224.npy")

train_images.shape, train_masks.shape

((2975, 224, 224, 3), (2975, 224, 224, 1))

In [7]:
import gc
gc.collect()

5735

In [ ]:
# Clear the session to avoid any potential issues with model state
tf.keras.backend.clear_session()

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train with ReduceLROnPlateau callback
history = model.fit(train_images, train_masks, epochs=50, batch_size=32, validation_split=0.2, callbacks=[reduce_lr])

Epoch 1/50


2024-09-07 05:05:06.364759: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-09-07 05:05:34.320748: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fc410ce69e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-09-07 05:05:34.320790: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-09-07 05:05:34.328661: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1725685534.440712  164513 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-09-07 05:05:39.899873: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.43GiB with freed_by_count=0. The caller indicates that this is no

75/75 [==============================] - 231s 2s/step - loss: 0.4708 - accuracy: 0.8520 - val_loss: 0.2846 - val_accuracy: 0.8666 - lr: 0.0010
Epoch 2/50


2024-09-07 05:08:53.386436: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.58GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


 4/75 [>.............................] - ETA: 1:31 - loss: 0.2911 - accuracy: 0.8684

...

12/75 [===>..........................] - ETA: 1:22 - loss: 0.2823 - accuracy: 0.8697

2024-09-07 05:09:09.146104: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.58GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


14/75 [====>.........................] - ETA: 1:19 - loss: 0.2805 - accuracy: 0.8711

2024-09-07 05:09:11.803505: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.58GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


33/75 [============>.................] - ETA: 55s - loss: 0.2743 - accuracy: 0.8719

In [ ]:
model.save('../model/attention_u_net_model/aunet_dynamic_original.keras')

In [12]:
2+2

4